In [0]:
!pip install scipy==1.1.0 Pillow==3.0.0

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
import sys
sys_path = '/content/drive/My Drive/'
sys.path.append(sys_path)

In [0]:
from __future__ import print_function
import random
import numpy as np
from data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Sreload external python modules
%load_ext autoreload
%autoreload 2

In [0]:
# Get CIFAR10
# !wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
# !tar -xzvf cifar-10-python.tar.gz
# !rm cifar-10-python.tar.gz 

In [0]:
# Load the raw CIFAR-10 data.
cifar10_dir = '/content/drive/My Drive/Colab Notebooks/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)


In [0]:
# Visualize some examples from the dataset.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [0]:
# Subsample the data
num_training = 5000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 500
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

In [0]:
# Reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

In [0]:
class KNearestNeighbor(object):
  """ a kNN classifier with L2 distance """

  def __init__(self):
    pass

  def train(self, X, y):
    
    self.X_train = X
    self.y_train = y
    
  def predict(self, X, k=1, num_loops=0):
    
    if num_loops == 0:
      dists = self.compute_distances_no_loops(X)
    elif num_loops == 1:
      dists = self.compute_distances_one_loop(X)
    elif num_loops == 2:
      dists = self.compute_distances_two_loops(X)
    else:
      raise ValueError('Invalid value %d for num_loops' % num_loops)

    return self.predict_labels(dists, k=k)

  def compute_distances_two_loops(self, X):
    
    num_test = X.shape[0]
    num_train = self.X_train.shape[0]
    dists = np.zeros((num_test, num_train))

    for i in range(num_test):
      for j in range(num_train):
           
        dists[i][j] = (np.sum((X[i] - self.X_train[j])**2))**0.5
        
    return dists

  def compute_distances_one_loop(self, X):
    
    num_test = X.shape[0]
    num_train = self.X_train.shape[0]
    dists = np.zeros((num_test, num_train))
    for i in range(num_test):
                
      dists[i] = np.sum((self.X_train - X[i])**2, axis=1)**0.5 #axis=1: by column 
    
    return dists

  def compute_distances_no_loops(self, X):
    
    num_test = X.shape[0]
    num_train = self.X_train.shape[0]
    dists = np.zeros((num_test, num_train)) 
    
    a = np.sum(X**2, axis=1)
    a = a.reshape(a.shape[0],1)
    b = np.sum(self.X_train**2, axis=1)
    b = b.reshape(1,b.shape[0])
    c = -2*np.dot(X,self.X_train.T)
    
    dists = (a + b + c )**0.5
    
    return dists

  def predict_labels(self, dists, k=1):
    
    num_test = dists.shape[0]
    y_pred = np.zeros(num_test)
    
    for i in range(num_test):
      testing_point = dists[i]
      sort_testing_point = np.argsort(testing_point)
      closest_y = np.array([])
      for idx_closest in range (0,k):
        closest_y = np.append(closest_y, self.y_train[sort_testing_point[idx_closest]])
          
      for idx in range (closest_y.shape[0]):
        # print (idx)
        if idx == 0: 
          y_pred[i] = closest_y[idx]
        else:
          (u,c) = np.unique(closest_y, return_counts=True)
          freq = np.asarray((u,c)) # row1: value, row2: freq
          for idx_freq in range(freq.shape[1]):
            if idx_freq == 0: 
              y_pred[i] = freq [0][idx_freq]
              max_cnt = freq [1][idx_freq]
            else:
              if max_cnt < freq [1][idx_freq]:
                y_pred[i] = freq [0][idx_freq]
              if (max_cnt == freq [1][idx_freq]) and (y_pred[i] > freq [0][idx_freq]):
                y_pred[i] = freq [0][idx_freq] 
      
    return y_pred



In [0]:
classifier = KNearestNeighbor()
classifier.train(X_train, y_train)

In [0]:
# Implement compute_distances_two_loops.

# Test implementation:
dists = classifier.compute_distances_two_loops(X_test)

In [0]:
# visualize the distance matrix: each row is a single test example and its distances to training examples
plt.imshow(dists, interpolation='none')
plt.show()

In [0]:
# implement the function predict_labels and run the code below:

# use k = 1 (which is Nearest Neighbor).
y_test_pred = classifier.predict_labels(dists, k=1)

# Compute and print the fraction of correctly predicted examples
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

In [0]:
# use k = 5 (which is Nearest Neighbor).

y_test_pred = classifier.predict_labels(dists, k=5)
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

In [0]:
# speed up distance matrix computation by using partial vectorization with one loop

# Implement the function compute_distances_one_loop:
dists_one = classifier.compute_distances_one_loop(X_test)

# Check (metrices)
difference = np.linalg.norm(dists - dists_one, ord='fro')
print('Difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [0]:
# Implement the fully vectorized version inside compute_distances_no_loops
dists_two = classifier.compute_distances_no_loops(X_test)

# Check (metrices)
difference = np.linalg.norm(dists - dists_two, ord='fro')
print('Difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [0]:
# Compare three implementations
def time_function(f, *args):
    """
    Call a function f with args and return the time (in seconds) that it took to execute.
    """
    import time
    tic = time.time()
    f(*args)
    toc = time.time()
    return toc - tic

two_loop_time = time_function(classifier.compute_distances_two_loops, X_test)
print('Two loop version took %f seconds' % two_loop_time)

one_loop_time = time_function(classifier.compute_distances_one_loop, X_test)
print('One loop version took %f seconds' % one_loop_time)

no_loop_time = time_function(classifier.compute_distances_no_loops, X_test)
print('No loop version took %f seconds' % no_loop_time)


In [0]:
## Cross-Validation

num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []

X_train_folds = np.array_split(X_train,num_folds)
y_train_folds = np.array_split(y_train,num_folds)

k_to_accuracies = {}

X_valid_slt = X_train_folds[num_folds-1] # last one as a validation set
y_valid_slt = y_train_folds[num_folds-1] # last one as a validation set

for k in range(len(k_choices)): 
  list_acc = []
  for idx_slt in range(num_folds-1):
    classifier = KNearestNeighbor()
    classifier.train(X_train_folds[idx_slt], y_train_folds[idx_slt])
    dists = classifier.compute_distances_no_loops(X_valid_slt)
    y_test_pred = classifier.predict_labels(dists, k_choices[k])
    num_correct = np.sum(y_test_pred == y_valid_slt)
    accuracy = float(num_correct) / num_test
    list_acc.append(accuracy)
  k_to_accuracies[k_choices[k]] = list_acc

# Print out the computed accuracies
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
      print('k = %d, accuracy = %f' % (k, accuracy))


In [0]:
# plot the raw observations
for k in k_choices:
    accuracies = k_to_accuracies[k]
    plt.scatter([k] * len(accuracies), accuracies)

# plot the trend line with error bars that correspond to standard deviation
accuracies_mean = np.array([np.mean(v) for k,v in sorted(k_to_accuracies.items())])
accuracies_std = np.array([np.std(v) for k,v in sorted(k_to_accuracies.items())])
plt.errorbar(k_choices, accuracies_mean, yerr=accuracies_std)
plt.title('Cross-validation on k')
plt.xlabel('k')
plt.ylabel('Cross-validation accuracy')
plt.show()